In [1]:
# pip install openpyxl

In [2]:
import pandas as pd
import openpyxl

In [3]:
sumula = pd.read_excel(
    r"C:\Users\José Roberto\Documents\Baygon Geral\Codigo\associacao-futebol\Teste_1_Jogador_Completo\Sumula_Ze.xlsx",
    sheet_name="Sumula",
    header=1,
    engine="calamine"
)
time = pd.read_excel(
    r"C:\Users\José Roberto\Documents\Baygon Geral\Codigo\associacao-futebol\Teste_1_Jogador_Completo\Times_Ze.xlsx",
    sheet_name="Times",
    header=1,
    engine="calamine"
)

In [4]:
time.columns

Index(['Baba', 'Colete', 'Boleiro1', 'Boleiro2', 'Boleiro3', 'Boleiro4',
       'Boleiro5', 'Boleiro6'],
      dtype='object')

In [5]:
duplas_final = pd.DataFrame()
for coluna_boleiro in time.columns:
    if "Boleiro" in coluna_boleiro:
        duplas = (sumula[["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"]]
                .merge(time, how="left", left_on=["Baba", "Colete", "Boleiro"], right_on=["Baba", "Colete", coluna_boleiro])
                .melt(id_vars=["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"],
            value_vars=["Boleiro1", "Boleiro2", "Boleiro3", "Boleiro4", "Boleiro5", "Boleiro6"],
            var_name="Origem",
            value_name="SegundoBoleiro")
            .drop(columns="Origem"))
        duplas = duplas[duplas["Boleiro"] != duplas["SegundoBoleiro"]].reset_index(drop=True)
        duplas_final = pd.concat([duplas_final, duplas], ignore_index=True)
duplas_final = duplas_final.rename(columns={"SegundoBoleiro":"Boleiro2"})

In [6]:
# Não está correto o merge por "Boleiro1", foi somente para facilitar/adiantar ## Código antigo
# duplas = (sumula[["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"]]
#           .merge(time, how="left", left_on=["Baba", "Colete", "Boleiro"], right_on=["Baba", "Colete", "Boleiro1"])
#           .melt(id_vars=["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"],
#     value_vars=["Boleiro1", "Boleiro2", "Boleiro3", "Boleiro4", "Boleiro5", "Boleiro6"],
#     var_name="Origem",
#     value_name="SegundoBoleiro")
#     .rename(columns={"SegundoBoleiro":"Boleiro2"})
#     .dropna(subset=["Boleiro2"])
#     .drop(columns="Origem"))

# duplas = duplas[duplas["Boleiro"] != duplas["Boleiro2"]].reset_index(drop=True)

In [ ]:
grupo_duplas = duplas_final.groupby(["Boleiro", "Boleiro2"]).agg(
    Total_Pizza=("Pizza", "sum"),
    Total_Vitorias=("Vitorias", "sum"),
    Total_Derrotas=("Derrotas", "sum"),
    Total_Empates=("Empates", "sum"),
    Total_Babas=("Baba", "count")
).reset_index()
grupo_duplas["Total_Partidas"] = grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"] + grupo_duplas["Total_Empates"]
grupo_duplas["P_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_VD"] = grupo_duplas["Total_Vitorias"] / (grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"])
grupo_duplas["M_Pizza"] = grupo_duplas["Total_Pizza"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Babas"]
grupo_duplas = grupo_duplas[
            [
                "Boleiro",
                "Boleiro2",
                "Total_Pizza",
                "Total_Vitorias",
                "Total_Derrotas",
                "Total_Empates",
                "Total_Partidas",
                "Total_Babas",
                "P_Vitorias",
                "P_Derrotas",
                "P_Empates",
                "P_VD",
                "M_Pizza",
                "M_Vitorias",
                "M_Derrotas",
                "M_Empates"
            ]
        ]



In [8]:
# grupo_duplas = grupo_duplas.sort_values("P_Vitorias", ascending=False).reset_index(drop=True)
# grupo_duplas = grupo_duplas.sort_values("P_Derrotas", ascending=False).reset_index(drop=True)
# grupo_duplas = grupo_duplas.sort_values("P_VD", ascending=False).reset_index(drop=True)
grupo_duplas = grupo_duplas.sort_values(["Total_Babas", "Total_Partidas", "Total_Vitorias"], ascending=False).reset_index(drop=True)
# grupo_duplas = grupo_duplas.sort_values(["P_Pizza", "P_Vitorias"], ascending=False).reset_index(drop=True)
grupo_duplas.index = grupo_duplas.index + 1
grupo_duplas.to_excel("grupo_duplas.xlsx", index=False)
# grupo_duplas

In [15]:
sumula.columns

Index(['Baba', 'Boleiro', 'Colete', 'Pontuacao', 'Gols', 'Assistencias',
       'Vitorias', 'Derrotas', 'Empates', 'Pontos no Baba', 'Mensalidade',
       'Colaboracao', 'Pontos de Copa', 'Pizza', 'Ouros', 'Gols Contra'],
      dtype='object')

In [ ]:
ranking_completo = sumula.groupby("Boleiro").agg(
    Artilharia=("Gols", "sum"),
    Assistencias=("Assistencias", "sum"),
    Babas_Conquistados=("Vitorias", "sum"),
    Babas_Perdidos=("Derrotas", "sum"),
    Empates=("Empates", "sum"),
    Pizza=("Pizza", "sum"),
    Presenca_no_Baba=("Baba", "count"),
    PB=("Pontos no Baba", "sum"),
    Mensalidade=("Mensalidade", "sum"),
    Pontos_de_Colaboracao=("Colaboracao","sum"),
    Copa=("Pontos de Copa","sum"),
    Ouros=("Ouros","sum"),
    Gols_Contra=("Gols Contra","sum"),
    ).reset_index()
# ranking_completo

,Boleiro,Artilharia,Assistencias,Babas_Conquistados,Babas_Perdidos,Empates,Pizza,Presenca_no_Baba,PB,Mensalidade,Pontos_de_Colaboracao,Copa,Ouros,Gols_Contra
0,Zé,72,72,107,108,53,6,37,555,33,70,35,3,3
